# Model performance

In [1]:
import numpy as np
import shap
import pickle
from datasets import load_dataset
from src.dataset_info import get_dataset_info
from transformers import pipeline, AutoTokenizer
import pandas as pd
from datasets import load_dataset, Dataset
import os
from tqdm import tqdm
from src.utils import token_segments, text_ft_index_ends

# from src.models import Model
import lightgbm as lgb
from src.models import WeightedEnsemble, StackModel, AllAsTextModel
from src.joint_masker import JointMasker
import argparse
import scipy as sp
from sklearn.metrics import roc_auc_score

/home/james/CodingProjects/TextNTabularExplanations/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def run_model(
    model_type, ds_type, test_set_size=100, tab_scale_factor=2, reverse=False
):
    di = get_dataset_info(ds_type, model_type)
    # Data
    train_df = load_dataset(
        di.ds_name, split="train", download_mode="force_redownload"
    ).to_pandas()
    y_train = train_df[di.label_col]

    test_df = load_dataset(
        di.ds_name, split="test", download_mode="force_redownload"
    ).to_pandas()
    test_df_sample = test_df.sample(test_set_size, random_state=55)

    # Models
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
    if model_type in [
        "all_text",
        "all_as_text_tnt_reorder",
        "all_as_text_base_reorder",
    ]:
        model_name = di.text_model_name
        # Define how to convert all columns to a single string
        if model_type == "all_text":

            def cols_to_str_fn(array):
                return " | ".join(
                    [
                        f"{col}: {val}"
                        for col, val in zip(di.tab_cols + di.text_cols, array)
                    ]
                )

        elif model_type == "all_as_text_tnt_reorder":
            # Reorder based on the new index order in di
            def cols_to_str_fn(array):
                original = di.tab_cols + di.text_cols
                new = di.tnt_reorder_cols if not reverse else di.tnt_reorder_cols[::-1]
                new_array = np.array([array[original.index(x)] for x in new])
                return " | ".join([f"{col}: {val}" for col, val in zip(new, new_array)])

            num = 2 if not reverse else 4
            model_name = model_name[:-1] + num
        else:
            # Reorder based on the new index order in di
            def cols_to_str_fn(array):
                original = di.tab_cols + di.text_cols
                new = di.tnt_reorder_cols if not reverse else di.tnt_reorder_cols[::-1]
                new_array = np.array([array[original.index(x)] for x in new])
                return " | ".join([f"{col}: {val}" for col, val in zip(new, new_array)])

            num = 1 if not reverse else 3
            model_name = model_name[:-1] + num

        text_pipeline = pipeline(
            "text-classification",
            model=di.text_model_name,
            tokenizer=tokenizer,
            device="cuda:0",
            truncation=True,
            padding=True,
            top_k=None,
        )
        model = AllAsTextModel(
            text_pipeline=text_pipeline,
            cols_to_str_fn=cols_to_str_fn,
            # cols=di.tab_cols + di.text_cols
        )
    else:
        text_pipeline = pipeline(
            "text-classification",
            model=di.text_model_name,
            tokenizer=tokenizer,
            device="cuda:0",
            truncation=True,
            padding=True,
            top_k=None,
        )
        # Define how to convert the text columns to a single string
        if len(di.text_cols) == 1:

            def cols_to_str_fn(array):
                return array[0]

        else:

            def cols_to_str_fn(array):
                return " | ".join(
                    [f"{col}: {val}" for col, val in zip(di.text_cols, array)]
                )

        # LightGBM requires explicitly marking categorical features
        train_df[di.categorical_cols] = train_df[di.categorical_cols].astype("category")
        test_df_sample[di.categorical_cols] = test_df_sample[
            di.categorical_cols
        ].astype("category")

        tab_model = lgb.LGBMClassifier(random_state=42)
        tab_model.fit(train_df[di.tab_cols], y_train)

        if model_type == "ensemble_50":
            model = WeightedEnsemble(
                tab_model=tab_model,
                text_pipeline=text_pipeline,
                text_weight=0.5,
                cols_to_str_fn=cols_to_str_fn,
            )
        elif model_type == "ensemble_75":
            model = WeightedEnsemble(
                tab_model=tab_model,
                text_pipeline=text_pipeline,
                text_weight=0.75,
                cols_to_str_fn=cols_to_str_fn,
            )
        elif model_type == "ensemble_25":
            model = WeightedEnsemble(
                tab_model=tab_model,
                text_pipeline=text_pipeline,
                text_weight=0.25,
                cols_to_str_fn=cols_to_str_fn,
            )
        elif model_type == "stack":
            """
            For the stack model, we make predictions on the validation set. These predictions
            are then used as features for the stack model (another LightGBM model) along with
            the other tabular features. In doing so the stack model learns, depending on the
            tabular features, when to trust the tabular model and when to trust the text model.
            """
            val_df = load_dataset(
                di.ds_name, split="validation", download_mode="force_redownload"
            ).to_pandas()
            val_df[di.categorical_cols] = val_df[di.categorical_cols].astype("category")
            y_val = val_df[di.label_col]
            val_text = list(map(cols_to_str_fn, val_df[di.text_cols].values))

            # Training set is the preditions from the tabular and text models on the validation set
            # plus the tabular features from the validation set
            text_val_preds = text_pipeline(val_text)
            # text_val_preds = np.array(
            #     [format_text_pred(pred) for pred in text_val_preds]
            # )
            text_val_preds = np.array(
                [[lab["score"] for lab in pred] for pred in text_val_preds]
            )

            # add text and tabular predictions to the val_df
            stack_val_df = val_df[di.tab_cols]
            tab_val_preds = tab_model.predict_proba(stack_val_df)
            for i in range(text_val_preds.shape[1]):
                stack_val_df[f"text_pred_{i}"] = text_val_preds[:, i]
            for i in range(tab_val_preds.shape[1]):
                stack_val_df[f"tab_pred_{i}"] = tab_val_preds[:, i]

            stack_model = lgb.LGBMClassifier(random_state=42)
            stack_model.fit(stack_val_df, y_val)

            model = StackModel(
                tab_model=tab_model,
                text_pipeline=text_pipeline,
                stack_model=stack_model,
                cols_to_str_fn=cols_to_str_fn,
            )
        else:
            raise ValueError(f"Invalid model type of {model_type}")

    np.random.seed(1)
    test_sample_vals = test_df_sample[di.tab_cols + di.text_cols].values
    test_vals = test_df[di.tab_cols + di.text_cols].values

    return (
        model.predict(test_sample_vals),
        test_df_sample[di.label_col].values,
        model.predict(test_vals),
        test_df[di.label_col].values,
    )


## Fake Job Postings (ROC-AUC)

In [3]:
ds_type = "fake"

for model_type in [
    "ensemble_25",
    "ensemble_50",
    "ensemble_75",
    "stack",
    "all_text",
    "all_as_text_tnt_reorder",
    # "all_as_text_base_reorder",
]:
    sample_preds, sample_y, preds, y = run_model(
        model_type, ds_type=ds_type, tab_scale_factor=1
    )

    print(f"\n{model_type} on {ds_type}")
    print(f"Test accuaracy: {np.mean(np.argmax(preds, axis=1) == y)}")
    print(
        f"Test accuaracy (sample): {np.mean(np.argmax(sample_preds, axis=1) == sample_y)}"
    )
    print(f"Test AUC: {roc_auc_score(y, preds[:, 1])}")
    print(f"Test AUC (sample): {roc_auc_score(sample_y, sample_preds[:, 1])}")
    print(f"Test % by label: {[np.mean(y == label) for label in np.unique(y)]}")
    print(
        f"Test % by label (sample): {[np.mean(sample_y == label) for label in np.unique(y)]}"
    )

for model_type in [
    "all_as_text_tnt_reorder",
    # "all_as_text_base_reorder",
]:
    sample_preds, sample_y, preds, y = run_model(
        model_type, ds_type=ds_type, tab_scale_factor=1, reverse=True
    )

    print(f"\n{model_type} reverse on {ds_type}")
    print(f"Test accuaracy: {np.mean(np.argmax(preds, axis=1) == y)}")
    print(
        f"Test accuaracy (sample): {np.mean(np.argmax(sample_preds, axis=1) == sample_y)}"
    )
    print(f"Test AUC: {roc_auc_score(y, preds[:, 1])}")
    print(f"Test AUC (sample): {roc_auc_score(sample_y, sample_preds[:, 1])}")
    print(f"Test % by label: {[np.mean(y == label) for label in np.unique(y)]}")
    print(
        f"Test % by label (sample): {[np.mean(sample_y == label) for label in np.unique(y)]}"
    )


Using dataset fake, ordinal version


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2968.37it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--fake_job_postings2_ordinal-d873cc356e36f3d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2599.78it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--fake_job_postings2_ordinal-d873cc356e36f3d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


...

Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2955.12it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--fake_job_postings2_ordinal-d873cc356e36f3d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2947.51it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--fake_job_postings2_ordinal-d873cc356e36f3d4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


KeyboardInterrupt: 

## IMDB  (ROC-AUC)

In [ ]:
ds_type = "imdb_genre"

for model_type in [
    "ensemble_25",
    "ensemble_50",
    "ensemble_75",
    "stack",
    "all_text",
    "all_as_text_tnt_reorder",
    "all_as_text_base_reorder",
]:
    sample_preds, sample_y, preds, y = run_model(
        model_type, ds_type=ds_type, tab_scale_factor=1
    )

    print(f"\n{model_type} on {ds_type}")
    print(f"Test accuaracy: {np.mean(np.argmax(preds, axis=1) == y)}")
    print(
        f"Test accuaracy (sample): {np.mean(np.argmax(sample_preds, axis=1) == sample_y)}"
    )
    print(f"Test AUC: {roc_auc_score(y, preds[:, 1])}")
    print(f"Test AUC (sample): {roc_auc_score(sample_y, sample_preds[:, 1])}")
    print(f"Test % by label: {[np.mean(y == label) for label in np.unique(y)]}")
    print(
        f"Test % by label (sample): {[np.mean(sample_y == label) for label in np.unique(y)]}"
    )

for model_type in [
    "all_as_text_tnt_reorder",
    "all_as_text_base_reorder",
]:
    sample_preds, sample_y, preds, y = run_model(
        model_type, ds_type=ds_type, tab_scale_factor=1, reverse=True
    )

    print(f"\n{model_type} reverse on {ds_type}")
    print(f"Test accuaracy: {np.mean(np.argmax(preds, axis=1) == y)}")
    print(
        f"Test accuaracy (sample): {np.mean(np.argmax(sample_preds, axis=1) == sample_y)}"
    )
    print(f"Test AUC: {roc_auc_score(y, preds[:, 1])}")
    print(f"Test AUC (sample): {roc_auc_score(sample_y, sample_preds[:, 1])}")
    print(f"Test % by label: {[np.mean(y == label) for label in np.unique(y)]}")
    print(
        f"Test % by label (sample): {[np.mean(sample_y == label) for label in np.unique(y)]}"
    )

Using dataset imdb_genre, ordinal version


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2163.87it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--imdb_genre_prediction_ordinal-95c476e18d2d7064/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2989.53it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--imdb_genre_prediction_ordinal-95c476e18d2d7064/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


...

Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--imdb_genre_prediction_ordinal-95c476e18d2d7064/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.

all_as_text_tnt_reorder on imdb_genre
Test accuaracy: 0.625
Test accuaracy (sample): 0.64
Test AUC: 0.8062256030427385
Test AUC (sample): 0.8028282828282829
Test % by label: [0.485, 0.515]
Test % by label (sample): [0.55, 0.45]
Using dataset imdb_genre, ordinal version


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 3003.08it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--imdb_genre_prediction_ordinal-95c476e18d2d7064/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 3036.42it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--imdb_genre_prediction_ordinal-95c476e18d2d7064/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.

all_as_text_base_reorder on imdb_genre
Test accuaracy: 0.645
Test accuaracy (sample): 0.66
Test AUC: 0.8113301971774597
Test AUC (sample): 0.8084848484848485
Test % by label: [0.485, 0.515]
Test % by label (sample): [0.55, 0.45]


## Kickstarter  (ROC-AUC)

In [ ]:
ds_type = "kick"

for model_type in [
    "ensemble_25",
    "ensemble_50",
    "ensemble_75",
    "stack",
    "all_text",
    "all_as_text_tnt_reorder",
    "all_as_text_base_reorder",
]:
    sample_preds, sample_y, preds, y = run_model(
        model_type, ds_type=ds_type, tab_scale_factor=1
    )

    print(f"\n{model_type} on {ds_type}")
    print(f"Test accuaracy: {np.mean(np.argmax(preds, axis=1) == y)}")
    print(
        f"Test accuaracy (sample): {np.mean(np.argmax(sample_preds, axis=1) == sample_y)}"
    )
    print(f"Test AUC: {roc_auc_score(y, preds[:, 1])}")
    print(f"Test AUC (sample): {roc_auc_score(sample_y, sample_preds[:, 1])}")
    print(f"Test % by label: {[np.mean(y == label) for label in np.unique(y)]}")
    print(
        f"Test % by label (sample): {[np.mean(sample_y == label) for label in np.unique(y)]}"
    )


for model_type in [
    "all_as_text_tnt_reorder",
    "all_as_text_base_reorder",
]:
    sample_preds, sample_y, preds, y = run_model(
        model_type, ds_type=ds_type, tab_scale_factor=1, reverse=True
    )

    print(f"\n{model_type} reverse on {ds_type}")
    print(f"Test accuaracy: {np.mean(np.argmax(preds, axis=1) == y)}")
    print(
        f"Test accuaracy (sample): {np.mean(np.argmax(sample_preds, axis=1) == sample_y)}"
    )
    print(f"Test AUC: {roc_auc_score(y, preds[:, 1])}")
    print(f"Test AUC (sample): {roc_auc_score(sample_y, sample_preds[:, 1])}")
    print(f"Test % by label: {[np.mean(y == label) for label in np.unique(y)]}")
    print(
        f"Test % by label (sample): {[np.mean(sample_y == label) for label in np.unique(y)]}"
    )


Using dataset kick, ordinal version


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2803.68it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--kick_starter_funding_ordinal-bc45dae77b1c676d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2799.94it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--kick_starter_funding_ordinal-bc45dae77b1c676d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


...

Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--kick_starter_funding_ordinal-bc45dae77b1c676d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.

all_as_text_tnt_reorder on kick
Test accuaracy: 0.711088504577823
Test accuaracy (sample): 0.69
Test AUC: 0.7393402976394993
Test AUC (sample): 0.676962676962677
Test % by label: [0.680384722093776, 0.319615277906224]
Test % by label (sample): [0.63, 0.37]
Using dataset kick, ordinal version


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2941.99it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--kick_starter_funding_ordinal-bc45dae77b1c676d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2829.53it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--kick_starter_funding_ordinal-bc45dae77b1c676d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.

all_as_text_base_reorder on kick
Test accuaracy: 0.7124294830296865
Test accuaracy (sample): 0.69
Test AUC: 0.7394847670821818
Test AUC (sample): 0.670956670956671
Test % by label: [0.680384722093776, 0.319615277906224]
Test % by label (sample): [0.63, 0.37]


## Jigsaw (ROC-AUC)

In [ ]:
ds_type = "jigsaw"

for model_type in [
    "ensemble_25",
    "ensemble_50",
    "ensemble_75",
    "stack",
    "all_text",
    "all_as_text_tnt_reorder",
    "all_as_text_base_reorder",
]:
    sample_preds, sample_y, preds, y = run_model(
        model_type, ds_type=ds_type, tab_scale_factor=1
    )

    print(f"\n{model_type} on {ds_type}")
    print(f"Test accuaracy: {np.mean(np.argmax(preds, axis=1) == y)}")
    print(
        f"Test accuaracy (sample): {np.mean(np.argmax(sample_preds, axis=1) == sample_y)}"
    )
    print(f"Test AUC: {roc_auc_score(y, preds[:, 1])}")
    print(f"Test AUC (sample): {roc_auc_score(sample_y, sample_preds[:, 1])}")
    print(f"Test % by label: {[np.mean(y == label) for label in np.unique(y)]}")
    print(
        f"Test % by label (sample): {[np.mean(sample_y == label) for label in np.unique(y)]}"
    )


for model_type in [
    "all_as_text_tnt_reorder",
    "all_as_text_base_reorder",
]:
    sample_preds, sample_y, preds, y = run_model(
        model_type, ds_type=ds_type, tab_scale_factor=1, reverse=True
    )

    print(f"\n{model_type} reverse on {ds_type}")
    print(f"Test accuaracy: {np.mean(np.argmax(preds, axis=1) == y)}")
    print(
        f"Test accuaracy (sample): {np.mean(np.argmax(sample_preds, axis=1) == sample_y)}"
    )
    print(f"Test AUC: {roc_auc_score(y, preds[:, 1])}")
    print(f"Test AUC (sample): {roc_auc_score(sample_y, sample_preds[:, 1])}")
    print(f"Test % by label: {[np.mean(y == label) for label in np.unique(y)]}")
    print(
        f"Test % by label (sample): {[np.mean(sample_y == label) for label in np.unique(y)]}"
    )


Using dataset jigsaw, ordinal version


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 3032.76it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--jigsaw_unintended_bias100K_ordinal-8e97391c4f489562/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 3066.01it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--jigsaw_unintended_bias100K_ordinal-8e97391c4f489562/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


...

Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 3002.37it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--jigsaw_unintended_bias100K_ordinal-8e97391c4f489562/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2986.69it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--jigsaw_unintended_bias100K_ordinal-8e97391c4f489562/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


100it [00:00, 420692.48it/s]
25000it [00:00, 444769.83it/s]


## Product Sentiment

In [ ]:
ds_type = "prod_sent"

for model_type in [
    "ensemble_25",
    "ensemble_50",
    "ensemble_75",
    "stack",
    "all_text",
    # "all_as_text_tnt_reorder",
    # "all_as_text_base_reorder",
]:
    sample_preds, sample_y, preds, y = run_model(
        model_type, ds_type=ds_type, tab_scale_factor=1
    )

    print(f"\n{model_type} on {ds_type}")
    print(f"Test accuaracy: {np.mean(np.argmax(preds, axis=1) == y)}")
    print(
        f"Test accuaracy (sample): {np.mean(np.argmax(sample_preds, axis=1) == sample_y)}"
    )
    # print(f"Test AUC: {roc_auc_score(y, preds[:, 1])}")
    # print(f"Test AUC (sample): {roc_auc_score(sample_y, sample_preds[:, 1])}")
    print(f"Test % by label: {[np.mean(y == label) for label in np.unique(y)]}")
    print(
        f"Test % by label (sample): {[np.mean(sample_y == label) for label in np.unique(y)]}"
    )

Using dataset prod_sent, ordinal version


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2763.05it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--product_sentiment_machine_hack_ordinal-bbe910d41184c5e7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2990.24it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--product_sentiment_machine_hack_ordinal-bbe910d41184c5e7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


...

Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--product_sentiment_machine_hack_ordinal-bbe910d41184c5e7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2945.44it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--product_sentiment_machine_hack_ordinal-bbe910d41184c5e7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100it [00:00, 907858.01it/s]
1273it [00:00, 1387204.21it/s]



stack on prod_sent
Test accuaracy: 0.6496465043205027
Test accuaracy (sample): 0.68
Test % by label: [0.010997643362136685, 0.0589159465828751, 0.5875883739198743, 0.3424980361351139]
Test % by label (sample): [0.01, 0.02, 0.62, 0.35]


## Wine

In [ ]:
ds_type = "wine"

for model_type in [
    "ensemble_25",
    "ensemble_50",
    "ensemble_75",
    "stack",
    "all_text",
    "all_as_text_tnt_reorder",
    # "all_as_text_base_reorder",
]:
    # sample_preds, sample_y, preds, y = run_model(
    #     model_type, ds_type=ds_type, tab_scale_factor=1
    # )
    preds, y = run_model(model_type, ds_type=ds_type, tab_scale_factor=1)

    print(f"\n{model_type} on {ds_type}")
    print(f"Test accuaracy: {np.mean(np.argmax(preds, axis=1) == y)}")
    print(
        f"Test accuaracy (sample): {np.mean(np.argmax(sample_preds, axis=1) == sample_y)}"
    )
    # print(f"Test AUC: {roc_auc_score(y, preds[:, 1])}")
    # print(f"Test AUC (sample): {roc_auc_score(sample_y, sample_preds[:, 1])}")
    print(f"Test % by label: {[np.mean(y == label) for label in np.unique(y)]}")
    print(
        f"Test % by label (sample): {[np.mean(sample_y == label) for label in np.unique(y)]}"
    )


for model_type in [
    "all_as_text_tnt_reorder",
    # "all_as_text_base_reorder",
]:
    sample_preds, sample_y, preds, y = run_model(
        model_type, ds_type=ds_type, tab_scale_factor=1, reverse=True
    )

    print(f"\n{model_type} reverse on {ds_type}")
    print(f"Test accuaracy: {np.mean(np.argmax(preds, axis=1) == y)}")
    print(
        f"Test accuaracy (sample): {np.mean(np.argmax(sample_preds, axis=1) == sample_y)}"
    )
    print(f"Test AUC: {roc_auc_score(y, preds[:, 1])}")
    print(f"Test AUC (sample): {roc_auc_score(sample_y, sample_preds[:, 1])}")
    print(f"Test % by label: {[np.mean(y == label) for label in np.unique(y)]}")
    print(
        f"Test % by label (sample): {[np.mean(sample_y == label) for label in np.unique(y)]}"
    )

Using dataset wine, ordinal version


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 1963.63it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--wine_reviews_ordinal-c036c1f97c7da848/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2816.23it/s]


Dataset parquet downloaded and prepared to /home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--wine_reviews_ordinal-c036c1f97c7da848/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


100it [00:00, 645277.54it/s]
21031it [00:00, 470854.79it/s]


ValueError: too many values to unpack (expected 2)

In [ ]:
y


array([14, 29,  5, ...,  5,  3, 26])

In [ ]:
np.argmax(preds, axis=1)


array([ 9,  9, 23, ..., 23,  2, 15])